<p style="font-weight:bold;"> <span style="font-size: 36px"> Utility Methods for Systemorph Initialization </span> </p>

In [0]:
using Systemorph.Vertex.DataSource;

# Files by Data Node

In [0]:
static readonly Dictionary<string,(string FileName, string Format)[]> DataNodeFiles = new Dictionary<string,(string FileName, string Format)[]>(){
    {"ES", new (string, string)[]{("DataNodes/DataNodes_ES.csv", ImportFormats.DataNode), 
                                ("DataNodes/DataNodeStates_ES_2020_12.csv", ImportFormats.DataNodeState), 
                                ("DataNodes/DataNodeParameters_ES_2020_12.csv", ImportFormats.DataNodeParameter)}}, 
    {"FR", new (string, string)[]{("DataNodes/DataNodes_FR.csv", ImportFormats.DataNode), 
                                ("DataNodes/DataNodeStates_FR_2020_12.csv", ImportFormats.DataNodeState), 
                                ("DataNodes/DataNodeParameters_FR_2020_12.csv", ImportFormats.DataNodeParameter)}}, 
    {"CH", new (string, string)[]{("DataNodes/DataNodes_CH.csv", ImportFormats.DataNode), 
                                ("DataNodes/DataNodeStates_CH_2020_12.csv", ImportFormats.DataNodeState),  
                                ("DataNodes/DataNodeParameters_CH_2020_12.csv", ImportFormats.DataNodeParameter)}}};

In [0]:
static readonly Dictionary<string, (string FileName, string Format)[]> TransactionalFilesByDataNode = new Dictionary<string, (string FileName, string Format)[]>(){
    {"ES", new (string, string)[]{("TransactionalData/Openings_ES_2020_12.csv", ImportFormats.Opening), 
                                ("TransactionalData/NominalCashflows_ES_2020_12.csv", ImportFormats.Cashflow), 
                                ("TransactionalData/Actuals_ES_2020_12.csv", ImportFormats.Actual)}}, 
    {"FR", new (string, string)[]{("TransactionalData/Openings_FR_2020_12.csv", ImportFormats.Opening), 
                                ("TransactionalData/NominalCashflows_FR_2020_12.csv", ImportFormats.Cashflow), 
                                ("TransactionalData/Actuals_FR_2020_12.csv", ImportFormats.Actual)}}, 
    {"CH", new (string, string)[]{("TransactionalData/Openings_CH_2020_12.csv", ImportFormats.Opening), 
                                ("TransactionalData/NominalCashflows_CH_2020_12.csv", ImportFormats.Cashflow),  
                                ("TransactionalData/Actuals_CH_2020_12.csv", ImportFormats.Actual), 
                                ("TransactionalData/SimpleValue_CH_2020_12.csv", ImportFormats.SimpleValue), 
                                ("TransactionalData/NominalCashflows_CH_2020_12_MTUP10pct.csv", ImportFormats.Cashflow), 
                                ("TransactionalData/NominalCashflows_CH_2021_3.csv", ImportFormats.Cashflow), 
                                ("TransactionalData/Actuals_CH_2021_3.csv", ImportFormats.Actual)}}};

# Import Tasks

In [0]:
public static async Task<ActivityLog[]> InitializeGenericAsync(this IImportVariable import, IDataSource dataSource, string pathToImport, 
                                                                Dictionary<string, (string FileName, string Format)[]> filesDictionary, params string[] reportingNodes){
    reportingNodes = !reportingNodes.Any() ? filesDictionary.Keys.ToArray() : reportingNodes;
    var activitylogs = Enumerable.Empty<ActivityLog>().ToArray();
    foreach(var reportingNode in reportingNodes){
        foreach(var pair in filesDictionary[reportingNode]){
            var currentActivityLog = await import.FromFile(pathToImport + pair.FileName).WithFormat(pair.Format).WithTarget(dataSource).WithActivityLog().ExecuteAsync();
            activitylogs = activitylogs.Append(currentActivityLog).ToArray();
        }
    }
    return activitylogs;
}

In [0]:
public static async Task<ActivityLog[]> InitializeDataNodesAsync(this IImportVariable import, IDataSource dataSource, string pathToImport, params string[] reportingNodes) =>
    await import.InitializeGenericAsync(dataSource, pathToImport, DataNodeFiles, reportingNodes);

In [0]:
public static async Task<ActivityLog[]> InitializeTransactionalDataAsync(this IImportVariable import, IDataSource dataSource, string pathToImport, params string[] reportingNodes) => 
    await import.InitializeGenericAsync(dataSource, pathToImport, TransactionalFilesByDataNode, reportingNodes);

In [0]:
public static async Task<ActivityLog[]> InitializeFullAsync(this IImportVariable import, IDataSource dataSource, string pathToImport, params string[] reportingNodes) =>
    (await import.InitializeDataNodesAsync(dataSource, pathToImport, reportingNodes)).Concat(await import.InitializeTransactionalDataAsync(dataSource, pathToImport, reportingNodes)).ToArray();